# Smoke Test — Robot Framework Fine-Tune Pipeline

**Purpose**: Validate the full code path in ~10 minutes before committing to a 90-minute full training run.

Uses `--smoke-test` mode: 10 training examples, 1 epoch, 3 eval samples, no HF upload.

**If this notebook prints `SMOKE TEST PASSED` → run the full training notebook.**
If it fails → fix the issue first (costs you 10 min, not 40).

In [ ]:
# Cell 1: Clone repo and install dependencies
import os, subprocess
repo_url = "https://github.com/arvind3/robot-finetune-model"
repo_dir = "repo"
if os.path.isdir(repo_dir):
    !rm -rf {repo_dir}
!git clone {repo_url} {repo_dir}
%cd {repo_dir}
!git rev-parse --short HEAD
!pip -q install -r requirements-train.txt
print('Repo ready')

In [ ]:
# Cell 2: Hugging Face authentication
import os
from getpass import getpass

token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN')
try:
    from google.colab import userdata
    token = token or userdata.get('HF_TOKEN')
except Exception:
    pass

if not token:
    token = getpass('Enter your Hugging Face token (HF_TOKEN): ')
os.environ['HF_TOKEN'] = token
print('HF_TOKEN set')

In [ ]:
# Cell 3: Dataset preparation (same as full run)
from pathlib import Path
from datasets import load_dataset
import subprocess, sys

dataset_repo = "arvind3/robotframework-expert-dataset"
train_path = Path('dataset/train.jsonl')
eval_path = Path('dataset/eval.jsonl')

if train_path.exists() and eval_path.exists():
    print('Using local dataset')
elif dataset_repo:
    print(f'Downloading dataset {dataset_repo}')
    ds = load_dataset(dataset_repo)
    Path('dataset').mkdir(exist_ok=True)
    ds['train'].to_json(train_path, orient='records', lines=True)
    ds['test' if 'test' in ds else 'validation' if 'validation' in ds else 'train'].to_json(eval_path, orient='records', lines=True)
else:
    proc = subprocess.run([sys.executable, 'tools/build_dataset.py', '--skip-validation'], check=True)

if not Path('data/eval_suite_v1.jsonl').exists():
    print('[WARN] data/eval_suite_v1.jsonl missing; evaluator will use dataset/eval.jsonl fallback')
print('Dataset ready')

In [ ]:
# Cell 4: Run smoke test
# Exercises the FULL pipeline: load model → train (10 examples, 1 epoch) →
# save adapter → merge → strip quantization_config → free VRAM → eval
# subprocess (3 samples, adapter path) → score → report.
# Skips HF upload. Should complete in ~5-8 minutes.
import subprocess, sys

base_model = "Qwen/Qwen2.5-3B-Instruct"
fallback_model = "Qwen/Qwen2.5-1.5B-Instruct"
hf_repo = "arvind3/robotframework-expert-qwen2.5-3b-lora"

result = subprocess.run(
    [
        sys.executable, "train/train_unsloth.py",
        "--base-model", base_model,
        "--fallback-model", fallback_model,
        "--hf-repo", hf_repo,
        "--seed", "42",
        "--smoke-test",
    ],
    check=False,
)

if result.returncode == 0:
    print("\n" + "="*60)
    print("SMOKE TEST PASSED — safe to run the full training notebook.")
    print("="*60)
else:
    raise RuntimeError(
        f"SMOKE TEST FAILED (exit {result.returncode}).\n"
        "Fix the issue before running the full training notebook."
    )